In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Aniridia"
cohort = "GSE204791"

# Input paths
in_trait_dir = "../../input/GEO/Aniridia"
in_cohort_dir = "../../input/GEO/Aniridia/GSE204791"

# Output paths
out_data_file = "../../output/preprocess/Aniridia/GSE204791.csv"
out_gene_data_file = "../../output/preprocess/Aniridia/gene_data/GSE204791.csv"
out_clinical_data_file = "../../output/preprocess/Aniridia/clinical_data/GSE204791.csv"
json_path = "../../output/preprocess/Aniridia/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Altered regulation of mRNA and miRNA expression in epithelial and stromal tissue of keratoconus corneas [RNA]"
!Series_summary	"Purpose:  To evaluate conjunctival cell microRNA and mRNA expression in relation to observed phenotype and genotype of aniridia-associated keratopathy (AAK) in a cohort of subjects with congenital aniridia.  Methods:  Using impression cytology, bulbar conjunctival cells were sampled from 20 subjects with congenital aniridia and 20 age and sex-matched healthy control subjects. RNA was extracted and microRNA and mRNA analysis was performed using microarrays. Results were related to the presence and severity of AAK determined by a standardized clinical grading scale and to the genotype (PAX6 mutation?) determined by clinical genetics.   Results:  Of the 2549 microRNAs analyzed, 21 were differentially expressed relative to controls. Among these miR-204-5p, an inhibitor of corneal neovascularization, was downregulated 26.8-fol

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the background information, this dataset contains both mRNA and miRNA expression data
# The study is about "microRNA and mRNA expression analysis," which indicates gene expression data is available
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability and 2.2 Data Type Conversion

# For trait (Aniridia)
# Looking at disease status in row 2 (KC = keratoconus, healthy control)
trait_row = 2

def convert_trait(value):
    if pd.isna(value):
        return None
    value_lower = str(value).lower()
    if ':' in value_lower:
        value_lower = value_lower.split(':', 1)[1].strip()
    
    if 'kc' in value_lower or 'keratoconus' in value_lower:
        return 1  # Disease present
    elif 'healthy' in value_lower or 'control' in value_lower:
        return 0  # Disease absent
    else:
        return None

# For age - available in row 0
age_row = 0

def convert_age(value):
    if pd.isna(value):
        return None
    if ':' in value:
        age_str = value.split(':', 1)[1].strip()
        try:
            return float(age_str)
        except ValueError:
            return None
    return None

# For gender - available in row 1
gender_row = 1

def convert_gender(value):
    if pd.isna(value):
        return None
    value_lower = str(value).lower()
    if ':' in value_lower:
        value_lower = value_lower.split(':', 1)[1].strip()
    
    if value_lower == 'f' or value_lower == 'female':
        return 0
    elif value_lower == 'm' or value_lower == 'male':
        return 1
    else:
        return None

# 3. Save Metadata - Initial filtering
# Determine if trait data is available (trait_row is not None)
is_trait_available = trait_row is not None

# Validate and save cohort info (initial filtering)
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction (if trait_row is not None)
# Note: In this case, we're unable to perform clinical feature extraction
# because we don't have access to the properly formatted clinical data.
# The sample characteristics dictionary only shows unique values for each characteristic
# and cannot be directly converted to the expected clinical data format.

# We will print the information we've determined about the dataset
print(f"Gene expression data available: {is_gene_available}")
print(f"Trait data available: {is_trait_available}")
print(f"Age data available: {age_row is not None}")
print(f"Gender data available: {gender_row is not None}")


Gene expression data available: True
Trait data available: True
Age data available: True
Gender data available: True


### Step 3: Gene Data Extraction

In [4]:
# 1. Re-identify the SOFT and matrix files to ensure we have the correct paths
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Extract the gene expression data from the matrix file
gene_data = get_genetic_data(matrix_file)

# 3. Print the first 20 row IDs (gene or probe identifiers)
print("\nFirst 20 gene/probe identifiers:")
print(gene_data.index[:20])

# 4. Print the dimensions of the gene expression data
print(f"\nGene data dimensions: {gene_data.shape[0]} genes × {gene_data.shape[1]} samples")

# Note: we keep is_gene_available as True since we successfully extracted gene expression data
is_gene_available = True



First 20 gene/probe identifiers:
Index(['(+)E1A_r60_1', '(+)E1A_r60_3', '(+)E1A_r60_a104', '(+)E1A_r60_a107',
       '(+)E1A_r60_a135', '(+)E1A_r60_a20', '(+)E1A_r60_a22', '(+)E1A_r60_a97',
       '(+)E1A_r60_n11', '(+)E1A_r60_n9', '3xSLv1', 'A_19_P00315452',
       'A_19_P00315492', 'A_19_P00315493', 'A_19_P00315502', 'A_19_P00315506',
       'A_19_P00315518', 'A_19_P00315519', 'A_19_P00315529', 'A_19_P00315541'],
      dtype='object', name='ID')

Gene data dimensions: 58341 genes × 31 samples


### Step 4: Gene Identifier Review

In [5]:
# Examining the gene identifiers from the output
# These identifiers appear to be Agilent microarray probe IDs (starting with "A_19_P") 
# and control probes (like "(+)E1A_r60_1"), not standard human gene symbols

# These probe IDs will need to be mapped to standard gene symbols
requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. First get the file paths using geo_get_relevant_filepaths function
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 3. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['GE_BrightCorner', 'DarkCorner', 'A_21_P0014386', 'A_33_P3396872', 'A_33_P3267760'], 'CONTROL_TYPE': ['pos', 'pos', 'FALSE', 'FALSE', 'FALSE'], 'REFSEQ': [nan, nan, nan, 'NM_001105533', nan], 'GB_ACC': [nan, nan, nan, 'NM_001105533', nan], 'LOCUSLINK_ID': [nan, nan, nan, 79974.0, 54880.0], 'GENE_SYMBOL': [nan, nan, nan, 'CPED1', 'BCOR'], 'GENE_NAME': [nan, nan, nan, 'cadherin-like and PC-esterase domain containing 1', 'BCL6 corepressor'], 'UNIGENE_ID': [nan, nan, nan, 'Hs.189652', nan], 'ENSEMBL_ID': [nan, nan, nan, nan, 'ENST00000378463'], 'ACCESSION_STRING': [nan, nan, nan, 'ref|NM_001105533|gb|AK025639|gb|BC030538|tc|THC2601673', 'ens|ENST00000378463'], 'CHROMOSOMAL_LOCATION': [nan, nan, 'unmapped', 'chr7:120901888-120901947', 'chrX:39909128-39909069'], 'CYTOBAND': [nan, nan, nan, 'hs|7q31.31', 'hs|Xp11.4'], 'DESCRIPTION': [nan, nan, nan, 'Homo sapiens cadherin-like and PC-esterase domain containing 1 (CPED1), transcript variant 2, mRNA [NM_001105533

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Based on the preview, determine the columns for gene mapping
# From the preview, we can see 'ID' is the column with probe identifiers
# and 'GENE_SYMBOL' is the column with gene symbols
prob_col = 'ID'  
gene_col = 'GENE_SYMBOL'

# 2. Get a mapping dataframe by extracting the relevant columns
gene_mapping = get_gene_mapping(gene_annotation, prob_col, gene_col)

# 3. Apply the gene mapping to convert probe-level data to gene-level data
gene_data = apply_gene_mapping(gene_data, gene_mapping)

# Print information about the mapped data
print(f"Gene data after mapping: {gene_data.shape[0]} genes × {gene_data.shape[1]} samples")
print("First 10 gene symbols:")
print(gene_data.index[:10])


Gene data after mapping: 29222 genes × 31 samples
First 10 gene symbols:
Index(['A1BG', 'A1BG-AS1', 'A1CF', 'A1CF-2', 'A1CF-3', 'A2M', 'A2M-1',
       'A2M-AS1', 'A2ML1', 'A2MP1'],
      dtype='object', name='Gene')


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the index
gene_data_normalized = normalize_gene_symbols_in_index(gene_data)

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data_normalized.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")
print(f"Gene data after normalization: {gene_data_normalized.shape[0]} genes × {gene_data_normalized.shape[1]} samples")

# Extract clinical features first (missed in previous steps)
clinical_features = geo_select_clinical_features(
    clinical_data, 
    trait, 
    trait_row,
    convert_trait,
    age_row,
    convert_age,
    gender_row,
    convert_gender
)

# Save the clinical data
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
clinical_features.to_csv(out_clinical_data_file)
print(f"Clinical features saved to {out_clinical_data_file}")
print("Clinical features preview:")
print(preview_df(clinical_features))

# 2. Link clinical and genetic data
linked_data = geo_link_clinical_genetic_data(clinical_features, gene_data_normalized)
print(f"Linked data shape: {linked_data.shape}")

# 3. Handle missing values
linked_data = handle_missing_values(linked_data, trait_col=trait)
print(f"Data shape after handling missing values: {linked_data.shape}")

# 4. Determine if trait is biased
print("\nChecking for bias in the trait variable:")
is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)

# 5. Conduct final quality validation
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=True,
    is_biased=is_biased,
    df=linked_data,
    note="Dataset contains gene expression data for aniridia patients and healthy controls."
)

# 6. Save linked data if usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset deemed not usable for trait association studies, linked data not saved.")

Normalized gene data saved to ../../output/preprocess/Aniridia/gene_data/GSE204791.csv
Gene data after normalization: 20778 genes × 31 samples
Clinical features saved to ../../output/preprocess/Aniridia/clinical_data/GSE204791.csv
Clinical features preview:
{'GSM6193900': [1.0, 59.0, 0.0], 'GSM6193903': [1.0, 28.0, 1.0], 'GSM6193906': [1.0, 58.0, 0.0], 'GSM6193908': [1.0, 56.0, 1.0], 'GSM6193911': [0.0, 50.0, 0.0], 'GSM6193913': [0.0, 30.0, 1.0], 'GSM6193916': [0.0, 53.0, 0.0], 'GSM6193918': [0.0, 77.0, 1.0], 'GSM6193920': [1.0, 50.0, 0.0], 'GSM6193923': [1.0, 67.0, 1.0], 'GSM6193925': [1.0, 29.0, 0.0], 'GSM6193928': [1.0, 46.0, 1.0], 'GSM6193930': [0.0, 56.0, 0.0], 'GSM6193933': [0.0, 65.0, 1.0], 'GSM6193935': [0.0, 58.0, 0.0], 'GSM6193938': [0.0, 81.0, 1.0], 'GSM6193940': [1.0, 28.0, 1.0], 'GSM6193943': [1.0, 58.0, 0.0], 'GSM6193945': [1.0, 67.0, 1.0], 'GSM6193948': [1.0, 46.0, 1.0], 'GSM6193950': [0.0, 87.0, 0.0], 'GSM6193953': [0.0, 87.0, 1.0], 'GSM6193955': [0.0, 70.0, 0.0], 'GSM6

Data shape after handling missing values: (31, 20781)

Checking for bias in the trait variable:
For the feature 'Aniridia', the least common label is '0.0' with 15 occurrences. This represents 48.39% of the dataset.
The distribution of the feature 'Aniridia' in this dataset is fine.

Quartiles for 'Age':
  25%: 50.0
  50% (Median): 58.0
  75%: 67.0
Min: 28.0
Max: 87.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 15 occurrences. This represents 48.39% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.



Linked data saved to ../../output/preprocess/Aniridia/GSE204791.csv
